In [11]:
import pymongo
import datetime
import collections

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import scipy.stats
from collections import Counter
from sklearn import cluster

# Importing our datasets

In [12]:
referendum_df = pd.read_csv("data/EU-referendum-result-data.csv")
referendum_df.head()

,id,Region_Code,Region,Area_Code,Area,Electorate,ExpectedBallots,VerifiedBallotPapers,Pct_Turnout,Votes_Cast,...,Remain,Leave,Rejected_Ballots,No_official_mark,Voting_for_both_answers,Writing_or_mark,Unmarked_or_void,Pct_Remain,Pct_Leave,Pct_Rejected
0,108,E12000006,East,E06000031,Peterborough,120892,87474,87469,72.35,87469,...,34176,53216,77,0,32,7,38,39.11,60.89,0.09
1,109,E12000006,East,E06000032,Luton,127612,84633,84636,66.31,84616,...,36708,47773,135,0,85,0,50,43.45,56.55,0.16
2,112,E12000006,East,E06000033,Southend-on-Sea,128856,93948,93939,72.90,93939,...,39348,54522,69,0,21,0,48,41.92,58.08,0.07
3,113,E12000006,East,E06000034,Thurrock,109897,79969,79954,72.75,79950,...,22151,57765,34,0,8,3,23,27.72,72.28,0.04
4,110,E12000006,East,E06000055,Bedford,119530,86136,86136,72.06,86135,...,41497,44569,69,0,26,1,42,48.22,51.78,0.08


In [13]:
immigration_df = pd.read_csv("data/immigration_df.csv",index_col="Unnamed: 0")
immigration_df.head()

,Area_Code,Area,country,population_2001,births_2002,deaths_2002,natchange_2002,internal_in_2002,internal_out_2002,internal_net_2002,...,internal_out_2015,internal_net_2015,international_in_2015,international_out_2015,international_net_2015,other_change_2015,population_2015,international_in_total,international_net_total,population_change
0,E06000001,Hartlepool,E,90152,1017,1042,-25,2140,2301,-161,...,2281,-144,248,135,113,15,92493,2524,546,2438
1,E06000002,Middlesbrough,E,141233,1663,1396,267,5109,6737,-1628,...,7071,-887,1250,450,800,12,139509,15883,8376,-2114
2,E06000003,Redcar and Cleveland,E,139159,1336,1523,-187,3951,4259,-308,...,4019,195,202,109,93,25,135275,3501,40,-4117
3,E06000004,Stockton-on-Tees,E,183795,1938,1719,219,6269,5531,738,...,6659,-217,994,557,437,-23,194803,9454,1953,10324
4,E06000005,Darlington,E,97894,1102,1173,-71,3815,3521,294,...,3779,-36,347,407,-60,19,105389,4671,304,7473


# Merging both DataFrames

In [14]:
immigration_referendum_df = pd.merge(immigration_df,referendum_df, on="Area_Code")
immigration_referendum_df.head().transpose()

,0,1,2,3,4
Area_Code,E06000001,E06000002,E06000003,E06000004,E06000005
Area_x,Hartlepool,Middlesbrough,Redcar and Cleveland,Stockton-on-Tees,Darlington
country,E,E,E,E,E
population_2001,90152,141233,139159,183795,97894
births_2002,1017,1663,1336,1938,1102
deaths_2002,1042,1396,1523,1719,1173
natchange_2002,-25,267,-187,219,-71
internal_in_2002,2140,5109,3951,6269,3815
internal_out_2002,2301,6737,4259,5531,3521
internal_net_2002,-161,-1628,-308,738,294


# Exporting merged dataframe to a new CSV file

In [15]:
immigration_referendum_df.to_csv("data/immigration_referendum_df.csv")